In [ ]:
# Downgrade forçado e "travamento" para evitar reinstalação automática
!pip install numpy==1.24.4 opencv-python==4.8.1.78 opencv-python-headless==4.8.1.78 --force-reinstall --quiet
!pip install albumentations==1.3.1 matplotlib scikit-learn tqdm sympy==1.13.1 --quiet

# Reinicia o ambiente do Colab para aplicar corretamente a versão do NumPy
import os
os.kill(os.getpid(), 9)


In [41]:
# ========================================================================================
# 📦 IMPORTAÇÕES E CONFIGURAÇÃO - Execute após reiniciar o runtime
# ========================================================================================

# Importações básicas
import os
import json
import time
import random
import warnings
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pickle
from sklearn.model_selection import train_test_split, StratifiedKFold

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import torchvision.transforms as transforms

# OpenCV e Albumentations
import cv2
import albumentations as A
from albumentations.pytorch import ToTensorV2

warnings.filterwarnings('ignore')

# Seeds para reprodutibilidade
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Configuração de pastas
base_path = "/content/drive/MyDrive/GuilhermeAlmeida"
folders = ["fotos", "modelos", "resultados", "resultados/metricas", "cache"]

for folder in folders:
    os.makedirs(os.path.join(base_path, folder), exist_ok=True)

# Hardware
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f"✅ Sistema configurado")
print(f"💻 Device: {device}")
print(f"📂 Base: {base_path}")

# Verifica imagens (assumindo que já estão extraídas)
fotos_path = os.path.join(base_path, "fotos")
if os.path.exists(fotos_path):
    num_images = len([f for f in os.listdir(fotos_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
    print(f"📸 {num_images} imagens encontradas")
else:
    print("❌ Pasta de fotos não encontrada")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Sistema configurado
💻 Device: cuda
📂 Base: /content/drive/MyDrive/GuilhermeAlmeida
📸 2298 imagens encontradas


In [42]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# ========================================================================================
# 📊 DATASET AVANÇADO PARA EYE TRACKING - VERSÃO CORRIGIDA
# ========================================================================================

class AdvancedEyeTrackingDataset(Dataset):
    """Dataset avançado para eye tracking com pré-processamento melhorado e verificações anti-NaN"""

    def __init__(self, images_path, labels_path, transform=None, augment=False, target_size=(224, 224)):
        self.images_path = images_path
        self.transform = transform
        self.augment = augment
        self.target_size = target_size

        # Carrega as imagens e labels
        self.image_files = []
        self.labels = []

        # CAMINHOS CORRIGIDOS - Procura por imagens na pasta local
        print(f"🔍 Procurando imagens em: {images_path}")
        if os.path.exists(images_path):
            for file in sorted(os.listdir(images_path)):
                if file.endswith(('.jpg', '.jpeg', '.png')):
                    full_path = os.path.join(images_path, file)
                    self.image_files.append(full_path)

        print(f"📸 Encontradas {len(self.image_files)} imagens")

        # Carrega labels se existir arquivo de labels
        if os.path.exists(labels_path):
            print(f"📋 Carregando labels de: {labels_path}")
            with open(labels_path, 'r') as f:
                labels_data = json.load(f)
                # Verifica se é o formato novo ou antigo
                if 'labels' in labels_data:
                    self.labels = labels_data['labels']
                else:
                    self.labels = labels_data

                # CORREÇÃO CRÍTICA: Garante que self.labels seja sempre uma lista
                if not isinstance(self.labels, list):
                    print(f"⚠️ Labels não é uma lista, é: {type(self.labels)}. Convertendo...")
                    if hasattr(self.labels, 'values'):
                        # Se for um dict, pega os valores
                        self.labels = list(self.labels.values())
                    elif hasattr(self.labels, '__iter__'):
                        # Se for iterável, converte para lista
                        self.labels = list(self.labels)
                    else:
                        print("❌ Erro: Não foi possível converter labels para lista")
                        self.labels = []
                
                # CORREÇÃO ADICIONAL: Converte todos os labels para float se forem strings
                print("🔄 Convertendo labels para formato numérico...")
                converted_labels = []
                for i, label in enumerate(self.labels):
                    try:
                        if isinstance(label, (list, tuple)) and len(label) == 2:
                            # Converte cada coordenada para float
                            x = float(label[0])
                            y = float(label[1])
                            converted_labels.append([x, y])
                        else:
                            print(f"⚠️ Label {i} tem formato inválido: {label}")
                            converted_labels.append([0.5, 0.5])  # Default central
                    except (ValueError, TypeError, IndexError) as e:
                        print(f"⚠️ Erro ao converter label {i}: {label} -> {e}")
                        converted_labels.append([0.5, 0.5])  # Default central
                
                self.labels = converted_labels
                print(f"✅ Labels convertidos: {len(self.labels)} válidos")
                
        else:
            print("⚠️ Arquivo de labels não encontrado. Criando labels dummy...")
            # Gera labels dummy para teste baseados na posição da imagem
            self.labels = self._generate_dummy_labels()

        # VALIDAÇÃO CRUCIAL DOS DADOS
        print("🔍 Validando dados carregados...")
        self._validate_data()

        # Verifica consistência entre imagens e labels
        if len(self.image_files) != len(self.labels):
            min_length = min(len(self.image_files), len(self.labels))
            print(f"⚠️ Ajustando: {len(self.image_files)} imagens → {len(self.labels)} labels. Usando {min_length} amostras.")
            self.image_files = self.image_files[:min_length]
            # CORREÇÃO: Garante que self.labels seja uma lista antes do slice
            if isinstance(self.labels, list):
                self.labels = self.labels[:min_length]
            else:
                print(f"⚠️ Erro: self.labels não é uma lista: {type(self.labels)}")
                # Cria labels dummy se houver problema
                self.labels = [[0.5, 0.5] for _ in range(min_length)]

        print(f"✅ Dataset configurado: {len(self.image_files)} amostras")

        # AUGMENTAÇÕES MAIS CONSERVADORAS para evitar NaN
        self.heavy_augmentation = A.Compose([
            A.HorizontalFlip(p=0.3),  # Reduzido
            A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1, p=0.3),  # Mais conservador
            A.HueSaturationValue(hue_shift_limit=3, sat_shift_limit=5, val_shift_limit=3, p=0.2),  # Mais conservador
            A.GaussNoise(var_limit=(2.0, 10.0), mean=0, p=0.1),  # Reduzido
            A.GaussianBlur(blur_limit=1, p=0.1),  # Reduzido
            A.ShiftScaleRotate(shift_limit=0.02, scale_limit=0.02, rotate_limit=2, p=0.2),  # Mais conservador
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2()
        ])

        self.light_augmentation = A.Compose([
            A.HorizontalFlip(p=0.2),  # Reduzido
            A.RandomBrightnessContrast(brightness_limit=0.05, contrast_limit=0.05, p=0.2),  # Muito conservador
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2()
        ])

        self.no_augmentation = A.Compose([
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2()
        ])

        # Estatísticas do dataset
        self.compute_dataset_stats()

    def _validate_data(self):
        """Valida dados carregados com correção automática"""
        invalid_labels = []
        for i, label in enumerate(self.labels):
            try:
                # Garante que é uma lista/tupla com 2 elementos
                if not isinstance(label, (list, tuple)) or len(label) != 2:
                    invalid_labels.append(i)
                    continue
                
                # Converte para float se necessário e valida range
                x = float(label[0])
                y = float(label[1])
                
                # Verifica se está no range válido [0, 1]
                if not (0 <= x <= 1 and 0 <= y <= 1):
                    invalid_labels.append(i)
                    continue
                    
                # Verifica valores NaN ou Inf
                if np.isnan(x) or np.isnan(y) or np.isinf(x) or np.isinf(y):
                    invalid_labels.append(i)
                    continue
                    
                # Atualiza o label com valores float validados
                self.labels[i] = [float(x), float(y)]
                
            except (ValueError, TypeError, IndexError) as e:
                print(f"⚠️ Erro ao validar label {i}: {label} -> {e}")
                invalid_labels.append(i)

        if invalid_labels:
            print(f"⚠️ Encontrados {len(invalid_labels)} labels inválidos. Corrigindo...")
            for i in invalid_labels:
                # Substitui por label válido central
                self.labels[i] = [0.5, 0.5]

        print(f"✅ Validação concluída. {len(self.labels) - len(invalid_labels)} labels válidos.")

    def _generate_dummy_labels(self):
        """Gera labels dummy baseados na posição da imagem"""
        labels = []
        # Pontos de calibração em grade 3x3
        calibration_points = [
            (0.2, 0.2), (0.5, 0.2), (0.8, 0.2),  # Linha superior
            (0.2, 0.5), (0.5, 0.5), (0.8, 0.5),  # Linha central
            (0.2, 0.8), (0.5, 0.8), (0.8, 0.8)   # Linha inferior
        ]

        for i in range(len(self.image_files)):
            # Cicla através dos pontos de calibração
            point_idx = i % len(calibration_points)
            gaze_x, gaze_y = calibration_points[point_idx]

            # Adiciona pequena variação aleatória
            gaze_x += np.random.normal(0, 0.02)  # Ainda menor
            gaze_y += np.random.normal(0, 0.02)

            # Garante que está no range [0.1, 0.9] para evitar extremos
            gaze_x = np.clip(gaze_x, 0.1, 0.9)
            gaze_y = np.clip(gaze_y, 0.1, 0.9)

            labels.append([float(gaze_x), float(gaze_y)])

        return labels

    def compute_dataset_stats(self):
        """Computa estatísticas do dataset para normalização"""
        print("📊 Computando estatísticas do dataset...")
        if len(self.image_files) == 0:
            print("❌ Nenhuma imagem encontrada para calcular estatísticas!")
            return

        # Sample de imagens para calcular estatísticas
        sample_size = min(100, len(self.image_files))
        sample_indices = np.random.choice(len(self.image_files), sample_size, replace=False)

        pixel_means = []
        pixel_stds = []

        for idx in sample_indices:
            img_path = self.image_files[idx]
            image = cv2.imread(img_path)
            if image is not None:
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                image = cv2.resize(image, self.target_size)
                pixel_means.append(np.mean(image, axis=(0, 1)))
                pixel_stds.append(np.std(image, axis=(0, 1)))

        if pixel_means:
            self.mean = np.mean(pixel_means, axis=0) / 255.0
            self.std = np.mean(pixel_stds, axis=0) / 255.0
            print(f"📊 Estatísticas: Mean={self.mean}, Std={self.std}")
        else:
            self.mean = [0.485, 0.456, 0.406]
            self.std = [0.229, 0.224, 0.225]
            print("⚠️ Usando estatísticas padrão do ImageNet")

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        # Carrega imagem
        img_path = self.image_files[idx]
        image = cv2.imread(img_path)

        if image is None:
            print(f"❌ Erro ao carregar imagem: {img_path}")
            # Retorna imagem dummy e label central
            image = np.zeros((224, 224, 3), dtype=np.uint8)
            gaze_x, gaze_y = 0.5, 0.5
        else:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, self.target_size)

            # Pega o label correspondente
            if idx < len(self.labels):
                gaze_x, gaze_y = self.labels[idx]
            else:
                gaze_x, gaze_y = 0.5, 0.5  # Default central

        # VERIFICAÇÕES ANTI-NaN CRÍTICAS
        if np.any(np.isnan(image)) or np.any(np.isinf(image)):
            print(f"⚠️ NaN/Inf detectado na imagem {idx}. Substituindo...")
            image = np.clip(image, 0, 255).astype(np.uint8)

        if np.isnan(gaze_x) or np.isnan(gaze_y) or np.isinf(gaze_x) or np.isinf(gaze_y):
            print(f"⚠️ NaN/Inf detectado no label {idx}. Usando (0.5, 0.5)")
            gaze_x, gaze_y = 0.5, 0.5

        # Clipping final nos labels
        gaze_x = np.clip(float(gaze_x), 0.0, 1.0)
        gaze_y = np.clip(float(gaze_y), 0.0, 1.0)

        # Aplicar transformações
        if self.transform:
            if self.augment and random.random() < 0.7:
                # Usa augmentação mais leve
                transformed = self.light_augmentation(image=image)
            else:
                transformed = self.no_augmentation(image=image)
            image = transformed['image']
        else:
            # Transformação mínima
            image = torch.from_numpy(image).permute(2, 0, 1).float() / 255.0
            # Normalização ImageNet
            normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            image = normalize(image)

        # VERIFICAÇÃO FINAL ANTI-NaN
        if torch.any(torch.isnan(image)) or torch.any(torch.isinf(image)):
            print(f"⚠️ NaN/Inf detectado após transformações no item {idx}")
            image = torch.zeros(3, self.target_size[0], self.target_size[1])

        return image, torch.tensor([gaze_x, gaze_y], dtype=torch.float32)

In [44]:
# ========================================================================================
# 🏗️ COMPONENTES ARQUITETURAIS BÁSICOS
# ========================================================================================

class ResidualBlock(nn.Module):
    """Bloco residual para melhor fluxo de gradiente"""

    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        residual = x
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(residual)
        out = F.relu(out)
        return out

class SEBlock(nn.Module):
    """Squeeze-and-Excitation Block para atenção de canal"""

    def __init__(self, channel, reduction=16):
        super(SEBlock, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(channel, channel // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channel // reduction, channel, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y.expand_as(x)

In [45]:
class CBAM(nn.Module):
    """Convolutional Block Attention Module"""

    def __init__(self, channel, reduction=16, spatial_kernel=7):
        super(CBAM, self).__init__()
        # Channel attention
        self.channel_attention = SEBlock(channel, reduction)

        # Spatial attention
        self.spatial_attention = nn.Sequential(
            nn.Conv2d(2, 1, kernel_size=spatial_kernel, stride=1, padding=spatial_kernel // 2, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        # Channel attention
        x = self.channel_attention(x)

        # Spatial attention
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        attention = torch.cat([avg_out, max_out], dim=1)
        attention = self.spatial_attention(attention)

        return x * attention

In [46]:
class AdvancedEyeTrackingCNN(nn.Module):
    """Rede Neural Convolucional Avançada para Eye Tracking"""

    def __init__(self, dropout_rate=0.3, num_classes=2):
        super(AdvancedEyeTrackingCNN, self).__init__()

        # Entrada: 3x224x224

        # Stem - processamento inicial
        self.stem = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )

        # Blocos residuais com atenção
        self.layer1 = self._make_layer(64, 64, 2, stride=1)
        self.layer2 = self._make_layer(64, 128, 2, stride=2)
        self.layer3 = self._make_layer(128, 256, 2, stride=2)
        self.layer4 = self._make_layer(256, 512, 2, stride=2)

        # Attention modules
        self.cbam1 = CBAM(64)
        self.cbam2 = CBAM(128)
        self.cbam3 = CBAM(256)
        self.cbam4 = CBAM(512)

        # Global Average Pooling
        self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))

        # Multi-scale feature extraction
        self.multi_scale = nn.ModuleList([
            nn.Conv2d(512, 256, kernel_size=1),
            nn.Conv2d(512, 256, kernel_size=3, padding=1),
            nn.Conv2d(512, 256, kernel_size=5, padding=2),
        ])

        # Feature fusion
        self.feature_fusion = nn.Conv2d(768, 512, kernel_size=1)

        # Classificador com múltiplas cabeças
        self.classifier = nn.Sequential(
            nn.Linear(512, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate),

            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate),

            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate / 2),

            nn.Linear(256, num_classes)
        )

        # Cabeça auxiliar para regularização
        self.aux_classifier = nn.Sequential(
            nn.Linear(256, 128),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate),
            nn.Linear(128, num_classes)
        )

        # Inicialização dos pesos
        self._initialize_weights()

    def _make_layer(self, in_channels, out_channels, blocks, stride):
        layers = []
        layers.append(ResidualBlock(in_channels, out_channels, stride))
        for _ in range(1, blocks):
            layers.append(ResidualBlock(out_channels, out_channels))
        return nn.Sequential(*layers)

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

            elif isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

    def forward(self, x, return_aux=False):
        # Stem
        x = self.stem(x)

        # Blocos residuais com atenção
        x = self.layer1(x)
        x = self.cbam1(x)

        x = self.layer2(x)
        x = self.cbam2(x)

        x = self.layer3(x)
        x = self.cbam3(x)
        aux_features = x  # Para classificador auxiliar

        x = self.layer4(x)
        x = self.cbam4(x)

        # Multi-scale features
        ms_features = []
        for conv in self.multi_scale:
            ms_features.append(conv(x))

        # Concatena features multi-scale
        x = torch.cat(ms_features, dim=1)
        x = self.feature_fusion(x)

        # Global pooling
        x = self.global_avg_pool(x)
        x = x.view(x.size(0), -1)

        # Classificação principal
        main_output = torch.sigmoid(self.classifier(x))

        if return_aux and self.training:
            # Classificador auxiliar
            aux_x = self.global_avg_pool(aux_features)
            aux_x = aux_x.view(aux_x.size(0), -1)
            aux_output = torch.sigmoid(self.aux_classifier(aux_x))
            return main_output, aux_output

        return main_output

In [47]:
class EnsembleModel(nn.Module):
    """Ensemble de múltiplos modelos para melhor performance"""

    def __init__(self, num_models=3):
        super(EnsembleModel, self).__init__()
        self.models = nn.ModuleList([
            AdvancedEyeTrackingCNN(dropout_rate=0.2 + i * 0.1)
            for i in range(num_models)
        ])
        self.num_models = num_models

    def forward(self, x):
        outputs = []
        for model in self.models:
            outputs.append(model(x))

        # Média ponderada dos outputs
        weights = torch.softmax(torch.randn(self.num_models), dim=0).to(x.device)
        ensemble_output = sum(w * out for w, out in zip(weights, outputs))

        return ensemble_output


In [48]:
class MetricsHandler:
    """Classe auxiliar para manipular métricas e gráficos"""

    def __init__(self, results_path='/content/drive/MyDrive/GuilhermeAlmeida/resultados/'):
        self.results_path = results_path
        os.makedirs(os.path.join(self.results_path, 'metricas'), exist_ok=True)

    def save_advanced_metrics(self, metrics_data, filename):
        """Salva métricas avançadas no Google Drive"""
        metrics_path = os.path.join(self.results_path, 'metricas', filename)
        os.makedirs(os.path.dirname(metrics_path), exist_ok=True)

        with open(metrics_path, 'w') as f:
            json.dump(metrics_data, f, indent=2)

        print(f"📊 Métricas salvas no Google Drive: {metrics_path}")

    def plot_training_curves(self, train_losses, val_losses, train_accs, val_accs, learning_rates, save_path=None):
        """Plota curvas de treinamento avançadas e salva no Google Drive"""
        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

        epochs = range(1, len(train_losses) + 1)

        # Plot das perdas
        ax1.plot(epochs, train_losses, 'b-', label='Perda Treino', linewidth=2)
        ax1.plot(epochs, val_losses, 'r-', label='Perda Validação', linewidth=2)
        ax1.set_title('Curvas de Perda', fontsize=14, fontweight='bold')
        ax1.set_xlabel('Época')
        ax1.set_ylabel('Perda Combinada')
        ax1.legend()
        ax1.grid(True, alpha=0.3)

        # Plot das acurácias
        ax2.plot(epochs, train_accs, 'b-', label='Erro Treino', linewidth=2)
        ax2.plot(epochs, val_accs, 'r-', label='Erro Validação', linewidth=2)
        ax2.set_title('Erro Euclidiano', fontsize=14, fontweight='bold')
        ax2.set_xlabel('Época')
        ax2.set_ylabel('Distância Euclidiana')
        ax2.legend()
        ax2.grid(True, alpha=0.3)

        # Learning rate
        ax3.plot(epochs, learning_rates, 'g-', linewidth=2)
        ax3.set_title('Learning Rate', fontsize=14, fontweight='bold')
        ax3.set_xlabel('Época')
        ax3.set_ylabel('Learning Rate')
        ax3.set_yscale('log')
        ax3.grid(True, alpha=0.3)

        # Overfitting detection
        if len(train_losses) > 10:
            train_smooth = np.convolve(train_losses, np.ones(5) / 5, mode='valid')
            val_smooth = np.convolve(val_losses, np.ones(5) / 5, mode='valid')
            gap = val_smooth - train_smooth
            ax4.plot(range(3, len(gap) + 3), gap, 'purple', linewidth=2)
            ax4.axhline(y=0, color='black', linestyle='--', alpha=0.5)
            ax4.set_title('Gap Treino-Validação (Overfitting)', fontsize=14, fontweight='bold')
            ax4.set_xlabel('Época')
            ax4.set_ylabel('Val Loss - Train Loss')
            ax4.grid(True, alpha=0.3)

        plt.tight_layout()

        if save_path:
            # Salva no Google Drive
            full_save_path = os.path.join(self.results_path, save_path)
            os.makedirs(os.path.dirname(full_save_path), exist_ok=True)
            plt.savefig(full_save_path, dpi=300, bbox_inches='tight')
            print(f"📈 Gráfico salvo no Google Drive: {full_save_path}")

        plt.show()

In [49]:
def create_smart_labels_from_calibration():
    """Cria labels inteligentes baseados nos dados de calibração e padrões de captura"""
    # CAMINHOS DO GOOGLE DRIVE
    calibration_file = '/content/drive/MyDrive/GuilhermeAlmeida/cache/calibragem.json'

    # Se não existe calibração, cria dados padrão
    calibration_data = {}
    if os.path.exists(calibration_file):
        with open(calibration_file, 'r') as f:
            calibration_data = json.load(f)
        print("✅ Dados de calibração carregados do Google Drive")
    else:
        print("⚠️ Arquivo de calibração não encontrado no Google Drive. Usando padrões.")

    # CAMINHO CORRIGIDO - Google Drive
    captured_faces_path = "/content/drive/MyDrive/GuilhermeAlmeida/fotos"
    if not os.path.exists(captured_faces_path):
        print(f"❌ Pasta {captured_faces_path} não encontrada no Google Drive.")
        print("📁 Certifique-se de que o Google Drive está montado e que a pasta existe")
        return None

    # Coleta todas as imagens do Google Drive
    image_files = []
    for file in sorted(os.listdir(captured_faces_path)):
        if file.lower().endswith(('.jpg', '.jpeg', '.png')):
            # Usa caminho relativo para compatibilidade
            image_files.append(file)

    if not image_files:
        print("❌ Nenhuma imagem encontrada na pasta do Google Drive.")
        return None

    print(f"📸 Encontradas {len(image_files)} imagens para labeling no Google Drive")

    # Pontos de calibração típicos em grid 3x3
    calibration_points = [
        (0.1, 0.1), (0.5, 0.1), (0.9, 0.1),  # Linha superior
        (0.1, 0.5), (0.5, 0.5), (0.9, 0.5),  # Linha central
        (0.1, 0.9), (0.5, 0.9), (0.9, 0.9)   # Linha inferior
    ]

    # Cria labels para todas as imagens
    labels = []
    for i, img_file in enumerate(image_files):
        # Usa padrão circular através dos pontos de calibração
        point_idx = i % len(calibration_points)
        gaze_x, gaze_y = calibration_points[point_idx]

        # Adiciona variação pequena para simular movimento natural
        gaze_x += np.random.normal(0, 0.03)  # Variação menor
        gaze_y += np.random.normal(0, 0.03)

        # Garante que está no range [0, 1]
        gaze_x = np.clip(gaze_x, 0.05, 0.95)  # Evita bordas extremas
        gaze_y = np.clip(gaze_y, 0.05, 0.95)

        labels.append([float(gaze_x), float(gaze_y)])  # Assegura tipo float

    # Salva dados organizados no Google Drive
    dataset_info = {
        'images': image_files,  # Apenas nomes dos arquivos
        'labels': labels,
        'calibration_data': calibration_data,
        'num_samples': len(labels),
        'created_at': time.time(),
        'images_path': captured_faces_path  # Caminho base do Google Drive
    }

    # Salva no Google Drive
    labels_path = '/content/drive/MyDrive/GuilhermeAlmeida/smart_labels.json'

    # Cria diretório se não existir
    os.makedirs(os.path.dirname(labels_path), exist_ok=True)

    with open(labels_path, 'w') as f:
        json.dump(dataset_info, f, indent=2)

    print(f"✅ Labels inteligentes criados para {len(labels)} imagens")
    print(f"📂 Salvo no Google Drive: {labels_path}")

    # VALIDAÇÃO DOS LABELS
    print("🔍 Validando labels criados...")
    for i, label in enumerate(labels[:5]):  # Testa primeiros 5
        if not (0 <= label[0] <= 1 and 0 <= label[1] <= 1):
            print(f"⚠️ Label {i} fora do range: {label}")
        if np.isnan(label[0]) or np.isnan(label[1]):
            print(f"❌ Label {i} contém NaN: {label}")
    print("✅ Validação de labels concluída")

    return labels_path

def load_smart_dataset(dataset_path):
    """Carrega dataset com informações inteligentes do Google Drive"""
    with open(dataset_path, 'r') as f:
        dataset_info = json.load(f)

    # Reconstrói caminhos completos do Google Drive
    images_base_path = dataset_info.get('images_path', '/content/drive/MyDrive/GuilhermeAlmeida/fotos')
    image_files = dataset_info['images']

    # Se são apenas nomes, reconstrói caminhos completos do Google Drive
    if not os.path.isabs(image_files[0]):
        image_files = [os.path.join(images_base_path, img) for img in image_files]

    return image_files, dataset_info['labels']

def load_advanced_trained_model(model_path='/content/drive/MyDrive/GuilhermeAlmeida/modelos/melhor_modelo_eyetracking_CORRIGIDO.pth'):
    """Carrega um modelo avançado treinado do Google Drive"""
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Carrega checkpoint
    checkpoint = torch.load(model_path, map_location=device)

    # Cria modelo
    model = AdvancedEyeTrackingCNN()
    model.load_state_dict(checkpoint['model_state_dict'])
    model.to(device)
    model.eval()

    print(f"✅ Modelo carregado do epoch {checkpoint.get('epoch', 'desconhecido')}")
    print(f"🎯 Melhor acurácia: {checkpoint.get('best_metrics', {}).get('val_accuracy', 'N/A')}")

    return model, device

def predict_gaze_advanced(model, image, device):
    """Prediz coordenadas de gaze com modelo avançado"""
    # Pré-processamento
    if len(image.shape) == 3:
        image = cv2.resize(image, (224, 224), interpolation=cv2.INTER_LANCZOS4)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Normalização
        mean = np.array([0.485, 0.456, 0.406])
        std = np.array([0.229, 0.224, 0.225])
        image = image.astype(np.float32) / 255.0
        image = (image - mean) / std

        # Para tensor
        image = torch.from_numpy(image).permute(2, 0, 1).unsqueeze(0)

    image = image.to(device)

    with torch.no_grad():
        prediction = model(image)
        gaze_x, gaze_y = prediction[0].cpu().numpy()

    return float(gaze_x), float(gaze_y)

In [50]:
# ========================================================================================
# 🔧 VERSÃO CORRIGIDA DO TRAINER - LEARNING RATE FIXO E MÉTRICAS AJUSTADAS
# ========================================================================================

class FixedLRAdvancedEyeTrackingTrainer:
    """Classe corrigida para treinamento com LR fixo e métricas mais realistas"""

    def __init__(self, model, device, use_amp=False, save_path='/content/drive/MyDrive/GuilhermeAlmeida/modelos/', results_path='/content/drive/MyDrive/GuilhermeAlmeida/resultados/'):
        self.model = model.to(device)
        self.device = device
        self.use_amp = use_amp
        self.scaler = torch.cuda.amp.GradScaler(enabled=use_amp)

        # Caminhos do Google Drive
        self.save_path = save_path
        self.results_path = results_path

        # Cria diretórios necessários no Google Drive
        os.makedirs(self.save_path, exist_ok=True)
        os.makedirs(self.results_path, exist_ok=True)
        os.makedirs(os.path.join(self.results_path, 'metricas'), exist_ok=True)

        # Métricas de treinamento
        self.train_losses = []
        self.val_losses = []
        self.train_accuracies = []
        self.val_accuracies = []
        self.learning_rates = []
        self.best_metrics = {
            'val_loss': float('inf'),
            'val_accuracy': float('inf'),
            'epoch': 0
        }

    def combined_loss(self, predictions, targets, aux_predictions=None):
        """Loss combinada com verificação de NaN melhorada"""
        # Verificação rigorosa de entrada
        if torch.isnan(predictions).any() or torch.isinf(predictions).any():
            print("❌ ERRO: Predições contêm NaN/Inf!")
        self.scheduler.step()
        return torch.tensor(1.0, device=predictions.device, requires_grad=True)

        if torch.isnan(targets).any() or torch.isinf(targets).any():
            print("❌ ERRO: Targets contêm NaN/Inf!")
            return torch.tensor(1.0, device=predictions.device, requires_grad=True)

        # Clamp das predições para range válido
        predictions = torch.clamp(predictions, 0.001, 0.999)
        targets = torch.clamp(targets, 0.001, 0.999)

        # MSE Loss básico
        mse_loss = F.mse_loss(predictions, targets)

        # Verificação do resultado
        if torch.isnan(mse_loss) or torch.isinf(mse_loss):
            print("❌ ERRO: MSE Loss inválido!")
            return torch.tensor(0.5, device=predictions.device, requires_grad=True)

        # Adiciona pequena regularização para estabilidade
        l2_reg = 0.0001 * torch.mean(predictions ** 2)
        total_loss = mse_loss + l2_reg

        # Verificação final
        if torch.isnan(total_loss) or torch.isinf(total_loss):
            return torch.tensor(0.5, device=predictions.device, requires_grad=True)

        return total_loss

    def calculate_improved_metrics(self, predictions, targets):
        """Calcula métricas melhoradas com thresholds mais realistas"""
        if torch.isnan(predictions).any() or torch.isnan(targets).any():
            return {
                'euclidean_distance': float('inf'),
                'angular_error': float('inf'),
                'accuracy_10': 0.0,
                'accuracy_20': 0.0,
                'accuracy_30': 0.0
            }

        predictions = torch.clamp(predictions, 0.0, 1.0)
        targets = torch.clamp(targets, 0.0, 1.0)
        euclidean_dist = torch.sqrt(torch.sum((predictions - targets) ** 2, dim=1))
        mean_euclidean = torch.mean(euclidean_dist).item()

        try:
            pred_norm = F.normalize(predictions + 1e-8, p=2, dim=1)
            target_norm = F.normalize(targets + 1e-8, p=2, dim=1)
            cosine_sim = torch.sum(pred_norm * target_norm, dim=1)
            cosine_sim = torch.clamp(cosine_sim, -0.999, 0.999)
            angular_error = torch.acos(cosine_sim)
            mean_angular = torch.mean(angular_error).item()
        except:
            mean_angular = float('inf')

        accuracy_10 = torch.mean((euclidean_dist < 0.1).float()).item()
        accuracy_20 = torch.mean((euclidean_dist < 0.2).float()).item()
        accuracy_30 = torch.mean((euclidean_dist < 0.3).float()).item()

        return {
            'euclidean_distance': mean_euclidean,
            'angular_error': mean_angular,
            'accuracy_10': accuracy_10,
            'accuracy_20': accuracy_20,
            'accuracy_30': accuracy_30
        }

    def train_epoch(self, train_loader, optimizer):
        self.model.train()
        total_loss = 0
        total_metrics = {'euclidean_distance': 0, 'angular_error': 0, 'accuracy_10': 0, 'accuracy_20': 0, 'accuracy_30': 0}
        valid_batches = 0
        skipped_batches = 0

        pbar = tqdm(train_loader, desc="Training", leave=False)
        for batch_idx, (data, target) in enumerate(pbar):

            # Move para device
            data, target = data.to(self.device), target.to(self.device)

            # Verificação de dados de entrada
            if torch.isnan(data).any() or torch.isinf(data).any():
                skipped_batches += 1
                continue

            if torch.isnan(target).any() or torch.isinf(target).any():
                skipped_batches += 1
                continue

            optimizer.zero_grad()

            try:
                if self.use_amp:
                    with torch.cuda.amp.autocast():
                        output = self.model(data)

                        # Verificação da saída
                        if torch.isnan(output).any() or torch.isinf(output).any():
                            skipped_batches += 1
                            continue

                        loss = self.combined_loss(output, target)

                    # Verificação do loss antes do backward
                    if torch.isnan(loss) or torch.isinf(loss) or loss.item() == float('inf'):
                        skipped_batches += 1
                        continue

                    # Backward pass
                    self.scaler.scale(loss).backward()

                    # Verifica gradientes
                    grad_norm = 0
                    for param in self.model.parameters():
                        if param.grad is not None:
                            if torch.isnan(param.grad).any() or torch.isinf(param.grad).any():
                                skipped_batches += 1
                                break
                            grad_norm += param.grad.data.norm(2).item() ** 2
                    else:  # Só executa se não houve break
                        grad_norm = grad_norm ** 0.5

                        if grad_norm > 10.0:
                            # Apenas conta como skip sem print detalhado
                            pass

                        self.scaler.unscale_(optimizer)
                        torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
                        self.scaler.step(optimizer)
                        self.scaler.update()

                        # Calcula métricas
                        with torch.no_grad():
                            metrics = self.calculate_improved_metrics(output, target)
                            if not any(np.isinf(v) or np.isnan(v) for v in metrics.values()):
                                for key in total_metrics:
                                    total_metrics[key] += metrics[key]
                                total_loss += loss.item()
                                valid_batches += 1
                            else:
                                skipped_batches += 1

                else:
                    # Sem AMP
                    output = self.model(data)

                    if torch.isnan(output).any() or torch.isinf(output).any():
                        skipped_batches += 1
                        continue

                    loss = self.combined_loss(output, target)

                    if torch.isnan(loss) or torch.isinf(loss) or loss.item() == float('inf'):
                        skipped_batches += 1
                        continue

                    loss.backward()

                    # Verifica gradientes
                    grad_norm = torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
                    if torch.isnan(grad_norm) or grad_norm > 10.0:
                        skipped_batches += 1
                        continue

                    optimizer.step()

                    # Calcula métricas
                    with torch.no_grad():
                        metrics = self.calculate_improved_metrics(output, target)
                        if not any(np.isinf(v) or np.isnan(v) for v in metrics.values()):
                            for key in total_metrics:
                                total_metrics[key] += metrics[key]
                            total_loss += loss.item()
                            valid_batches += 1
                        else:
                            skipped_batches += 1

                # Atualiza progress bar
                if valid_batches > 0:
                    current_loss = total_loss / valid_batches
                    current_eucl = total_metrics['euclidean_distance'] / valid_batches
                    current_acc20 = total_metrics['accuracy_20'] / valid_batches

                    pbar.set_postfix({
                        'Loss': f"{current_loss:.4f}",
                        'Eucl': f"{current_eucl:.4f}",
                        'Acc20': f"{current_acc20:.3f}",
                        'Valid': f"{valid_batches}/{batch_idx+1}",
                        'Skip': f"{skipped_batches}"
                    })

            except Exception as e:
                skipped_batches += 1
                continue

        # Resultados finais da época (simplificado)
        if valid_batches > 0:
            avg_loss = total_loss / valid_batches
            for key in total_metrics:
                total_metrics[key] /= valid_batches
        else:
            avg_loss = float('inf')
            for key in total_metrics:
                total_metrics[key] = float('inf')

        return avg_loss, total_metrics

    def validate_epoch(self, val_loader):
        self.model.eval()
        total_loss = 0
        total_metrics = {'euclidean_distance': 0, 'angular_error': 0, 'accuracy_10': 0, 'accuracy_20': 0, 'accuracy_30': 0}
        valid_batches = 0

        with torch.no_grad():
            for data, target in tqdm(val_loader, desc="Validating", leave=False):
                data, target = data.to(self.device), target.to(self.device)
                if torch.isnan(data).any() or torch.isnan(target).any():
                    continue

                try:
                    if self.use_amp:
                        with torch.cuda.amp.autocast():
                            output = self.model(data)
                            loss = self.combined_loss(output, target)
                    else:
                        output = self.model(data)
                        loss = self.combined_loss(output, target)

                    if torch.isnan(loss) or torch.isinf(loss):
                        continue

                    metrics = self.calculate_improved_metrics(output, target)
                    if not any(np.isinf(v) or np.isnan(v) for v in metrics.values()):
                        for key in total_metrics:
                            total_metrics[key] += metrics[key]
                        total_loss += loss.item()
                        valid_batches += 1
                except Exception:
                    continue

        if valid_batches > 0:
            avg_loss = total_loss / valid_batches
            for key in total_metrics:
                total_metrics[key] /= valid_batches
        else:
            avg_loss = float('inf')
            for key in total_metrics:
                total_metrics[key] = float('inf')

        return avg_loss, total_metrics

    def train(self, train_loader, val_loader, epochs=100, learning_rate=0.0001, use_fixed_lr=True):
        optimizer = optim.Adam(self.model.parameters(), lr=learning_rate, weight_decay=1e-5, eps=1e-8)

        scheduler = None
        if not use_fixed_lr:
            scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=20, min_lr=1e-6, verbose=True)

        patience = 1000
        patience_counter = 0

        print(f"Iniciando treinamento CORRIGIDO por {epochs} épocas...")
        print(f"Device: {self.device}")
        print(f"Learning Rate: {learning_rate} (FIXO: {use_fixed_lr})")
        print(f"Mixed Precision: {self.use_amp}")
        print(f"Parâmetros do modelo: {sum(p.numel() for p in self.model.parameters()):,}")

        for epoch in range(epochs):
            print(f"\n{'=' * 20} Época {epoch + 1}/{epochs} {'=' * 20}")
            train_loss, train_metrics = self.train_epoch(train_loader, optimizer)
            val_loss, val_metrics = self.validate_epoch(val_loader)

            if scheduler:
                scheduler.step(val_loss)

            self.train_losses.append(train_loss)
            self.val_losses.append(val_loss)
            self.train_accuracies.append(train_metrics['euclidean_distance'])
            self.val_accuracies.append(val_metrics['euclidean_distance'])

            current_lr = optimizer.param_groups[0]['lr']
            self.learning_rates.append(current_lr)

            print(f"Train - Loss: {train_loss:.4f}, Eucl: {train_metrics['euclidean_distance']:.4f}")
            print(f"      - Acc@10%: {train_metrics['accuracy_10']:.3f}, Acc@20%: {train_metrics['accuracy_20']:.3f}, Acc@30%: {train_metrics['accuracy_30']:.3f}")
            print(f"Val   - Loss: {val_loss:.4f}, Eucl: {val_metrics['euclidean_distance']:.4f}")
            print(f"      - Acc@10%: {val_metrics['accuracy_10']:.3f}, Acc@20%: {val_metrics['accuracy_20']:.3f}, Acc@30%: {val_metrics['accuracy_30']:.3f}")
            print(f"Learning Rate: {current_lr:.2e}")

            # Plota gráficos a cada 10 épocas
            if (epoch + 1) % 10 == 0:
                try:
                    metrics_handler = MetricsHandler(self.results_path)
                    metrics_handler.plot_training_curves(
                        self.train_losses,
                        self.val_losses,
                        self.train_accuracies,
                        self.val_accuracies,
                        self.learning_rates,
                        f'curvas_epoca_{epoch + 1}.png'
                    )
                    print(f"📈 Gráficos salvos para época {epoch + 1}")
                except Exception as e:
                    print(f"⚠️ Erro ao salvar gráficos: {e}")

            if np.isinf(train_loss) or np.isnan(train_loss):
                print("❌ ERRO: Loss de treino inválido! Parando treinamento...")
                break

            if val_metrics['accuracy_20'] > self.best_metrics.get('best_acc_20', 0):
                self.best_metrics.update({
                    'val_loss': val_loss,
                    'val_accuracy': val_metrics['euclidean_distance'],
                    'best_acc_20': val_metrics['accuracy_20'],
                    'epoch': epoch + 1
                })
                patience_counter = 0
                best_model_path = os.path.join(self.save_path, 'melhor_modelo_eyetracking_CORRIGIDO.pth')
                torch.save({
                    'model_state_dict': self.model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'epoch': epoch + 1,
                    'best_metrics': self.best_metrics,
                    'train_losses': self.train_losses,
                    'val_losses': self.val_losses
                }, best_model_path)
                print(f"✓ Novo melhor modelo salvo! Acc@20%: {val_metrics['accuracy_20']:.3f}")
            else:
                patience_counter += 1

            if patience_counter >= patience:
                print(f"Early stopping após {epoch + 1} épocas")
                break

        print(f"\n{'=' * 60}")
        print("Treinamento CORRIGIDO concluído!")
        print(f"Melhor época: {self.best_metrics['epoch']}")
        best_acc_20 = self.best_metrics.get("best_acc_20", None)
        if best_acc_20 is not None:
            print(f"Melhor Acc@20%: {best_acc_20:.3f}")
        else:
            print("Melhor Acc@20%: N/A")

        return self.train_losses, self.val_losses, self.train_accuracies, self.val_accuracies

        # Scheduler: reduz o learning rate a cada 100 épocas (gamma = 0.1)
        self.scheduler = torch.optim.lr_scheduler.StepLR(self.optimizer, step_size=100, gamma=0.1)


In [51]:
# ========================================================================================
# 🔧 FUNÇÕES AUXILIARES PARA VERIFICAÇÃO E DEBUGGING
# ========================================================================================

def verify_model_initialization(model, device):
    """Verifica se o modelo está inicializado corretamente"""
    print("🔍 Verificando inicialização do modelo...")

    # Verifica se há parâmetros com valores inválidos
    invalid_params = 0
    total_params = 0

    for name, param in model.named_parameters():
        total_params += param.numel()
        if torch.isnan(param).any() or torch.isinf(param).any():
            print(f"❌ Parâmetro inválido encontrado: {name}")
            invalid_params += param.numel()
            # Reinicializa o parâmetro problemático
            if 'weight' in name:
                nn.init.xavier_normal_(param)
            elif 'bias' in name:
                nn.init.constant_(param, 0)

    if invalid_params > 0:
        print(f"⚠️ {invalid_params}/{total_params} parâmetros foram reinicializados")
    else:
        print("✅ Todos os parâmetros estão válidos")

    # Teste com entrada sintética
    print("🧪 Testando forward pass...")
    model.eval()
    with torch.no_grad():
        test_input = torch.randn(2, 3, 224, 224).to(device)
        try:
            test_output = model(test_input)
            if torch.isnan(test_output).any() or torch.isinf(test_output).any():
                print("❌ Forward pass produz valores inválidos!")
                return False
            else:
                print(f"✅ Forward pass OK - Output range: [{test_output.min():.3f}, {test_output.max():.3f}]")
                return True
        except Exception as e:
            print(f"❌ Erro no forward pass: {e}")
            return False

def create_safer_model(dropout_rate=0.2):
    """Cria um modelo com inicialização mais segura"""
    print("🏗️ Criando modelo com inicialização segura...")

    model = AdvancedEyeTrackingCNN(dropout_rate=dropout_rate)

    # Inicialização mais conservadora
    for module in model.modules():
        if isinstance(module, nn.Conv2d):
            # Inicialização He/Kaiming mais conservadora
            nn.init.kaiming_normal_(module.weight, mode='fan_out', nonlinearity='relu', a=0.1)
            if module.bias is not None:
                nn.init.constant_(module.bias, 0)
        elif isinstance(module, nn.BatchNorm2d):
            nn.init.constant_(module.weight, 1)
            nn.init.constant_(module.bias, 0)
        elif isinstance(module, nn.Linear):
            # Inicialização Xavier mais conservadora
            nn.init.xavier_normal_(module.weight, gain=0.5)
            if module.bias is not None:
                nn.init.constant_(module.bias, 0)

    print("✅ Modelo inicializado com configuração segura")
    return model

def debug_dataloader(dataloader, device, num_samples=3):
    """Debug detalhado do dataloader"""
    print(f"🔍 Analisando {num_samples} amostras do dataloader...")

    for i, (data, target) in enumerate(dataloader):
        if i >= num_samples:
            break

        print(f"\n--- Amostra {i+1} ---")
        print(f"Data shape: {data.shape}")
        print(f"Target shape: {target.shape}")
        print(f"Data range: [{data.min():.3f}, {data.max():.3f}]")
        print(f"Target range: [{target.min():.3f}, {target.max():.3f}]")

        # Verifica valores inválidos
        if torch.isnan(data).any():
            print("❌ Data contém NaN!")
            nan_count = torch.isnan(data).sum().item()
            print(f"   {nan_count} valores NaN encontrados")

        if torch.isinf(data).any():
            print("❌ Data contém Inf!")
            inf_count = torch.isinf(data).sum().item()
            print(f"   {inf_count} valores Inf encontrados")

        if torch.isnan(target).any():
            print("❌ Target contém NaN!")

        if torch.isinf(target).any():
            print("❌ Target contém Inf!")

        # Move para device e testa
        data, target = data.to(device), target.to(device)
        print(f"✅ Dados movidos para {device} com sucesso")

    print("🔍 Debug do dataloader concluído")

def fix_dataset_issues(dataset):
    """Tenta corrigir problemas conhecidos no dataset"""
    print("🔧 Verificando e corrigindo dataset...")

    # Testa algumas amostras
    problematic_indices = []

    for i in range(min(50, len(dataset))):
        try:
            data, target = dataset[i]

            if torch.isnan(data).any() or torch.isinf(data).any():
                problematic_indices.append(i)
                print(f"⚠️ Amostra {i}: dados problemáticos")

            if torch.isnan(target).any() or torch.isinf(target).any():
                problematic_indices.append(i)
                print(f"⚠️ Amostra {i}: target problemático")

        except Exception as e:
            problematic_indices.append(i)
            print(f"❌ Erro na amostra {i}: {e}")

    if problematic_indices:
        print(f"⚠️ Encontradas {len(problematic_indices)} amostras problemáticas de {50} testadas")
        return False
    else:
        print("✅ Dataset parece estar em bom estado")
        return True

def test_system():
    """Testa o sistema antes de iniciar o treinamento"""
    print("🧪 Testando funcionamento do sistema...")

    try:
        # Teste PyTorch básico
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        test_tensor = torch.randn(2, 3, 224, 224).to(device)
        print(f"✅ Tensor de teste criado: {test_tensor.shape} em {device}")

        # Teste transformações básicas
        test_transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        print("✅ Transforms funcionando")

        # Teste Albumentations
        test_albu = A.Compose([
            A.Resize(224, 224),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2()
        ])

        # Cria imagem de teste
        test_image = np.random.randint(0, 255, (100, 100, 3), dtype=np.uint8)
        transformed = test_albu(image=test_image)
        print("✅ Albumentations funcionando")

        # Teste OpenCV
        test_cv_image = np.zeros((100, 100, 3), dtype=np.uint8)
        resized = cv2.resize(test_cv_image, (224, 224))
        print("✅ OpenCV funcionando")

        # Teste operação neural básica
        test_conv = nn.Conv2d(3, 64, kernel_size=3, padding=1).to(device)
        output = test_conv(test_tensor[:1])  # Testa só 1 imagem
        print(f"✅ Operação neural: {output.shape}")

        print("\n🎉 TODOS OS TESTES PASSARAM!")
        print("🚀 Sistema pronto para treinamento!")
        return True

    except Exception as e:
        print(f"\n❌ ERRO no teste: {e}")
        print("💡 Possíveis soluções:")
        print("   1. Reinicie o runtime (Runtime → Restart Runtime)")
        print("   2. Execute as células de instalação novamente")
        print("   3. Verifique se está usando GPU no Colab")
        import traceback
        traceback.print_exc()
        return False

In [57]:
import json
import random

json_path = "/content/drive/MyDrive/GuilhermeAlmeida/smart_labels.json"

with open(json_path, "r") as f:
    data = json.load(f)

# Gerar labels aleatórias dentro do intervalo [0, 1]
corrected = []
for item in data:
    if isinstance(item, dict) and "image" in item:
        x = round(random.uniform(0.1, 0.9), 3)
        y = round(random.uniform(0.1, 0.9), 3)
        corrected.append({"image": item["image"], "label": [x, y]})

# Salvar as novas labels
with open(json_path, "w") as f:
    json.dump(corrected, f)

print(f"✅ {len(corrected)} labels aleatórias geradas e salvas.")


✅ 2298 labels aleatórias geradas e salvas.


In [62]:
import json

json_path = "/content/drive/MyDrive/GuilhermeAlmeida/smart_labels.json"

with open(json_path, "r") as f:
    data = json.load(f)

# Força conversão para float (mesmo se parecer número)
corrected = []
for item in data:
    try:
        x = float(item["label"][0])
        y = float(item["label"][1])
        corrected.append({"image": item["image"], "label": [x, y]})
    except:
        continue

with open(json_path, "w") as f:
    json.dump(corrected, f)

print(f"✅ {len(corrected)} labels corrigidas com floats reais.")


✅ 2298 labels corrigidas com floats reais.


In [63]:
import json

json_path = "/content/drive/MyDrive/GuilhermeAlmeida/smart_labels.json"

with open(json_path, "r") as f:
    data = json.load(f)

corrected = []
if isinstance(data, dict):
    for k, v in data.items():
        try:
            x = float(v[0])
            y = float(v[1])
            if 0 <= x <= 1 and 0 <= y <= 1:
                corrected.append({"image": k, "label": [x, y]})
        except:
            continue

elif isinstance(data, list):
    for item in data:
        try:
            x = float(item["label"][0])
            y = float(item["label"][1])
            if 0 <= x <= 1 and 0 <= y <= 1:
                corrected.append({"image": item["image"], "label": [x, y]})
        except:
            continue

with open(json_path, "w") as f:
    json.dump(corrected, f)

print(f"✅ Labels corrigidas: {len(corrected)} válidas salvas.")


✅ Labels corrigidas: 2298 válidas salvas.


In [ ]:
# ========================================================================================
# 🚀 FUNÇÃO PRINCIPAL DE TREINAMENTO SIMPLIFICADA
# ========================================================================================

def main_fixed_lr():
    """Função principal SIMPLIFICADA para treinar com learning rate fixo"""
    print("=" * 80)
    print("🔧 TREINAMENTO SIMPLIFICADO - FOCO NO PROBLEMA DO LOSS")
    print("=" * 80)

    # Configurações SIMPLIFICADAS
    CONFIG = {
        'BATCH_SIZE': 32,
        'EPOCHS': 300,
        'LEARNING_RATE': 0.0001,
        'VALIDATION_SPLIT': 0.15,
        'TARGET_SIZE': (224, 224),
        'USE_AMP': True,
        'NUM_WORKERS': 4,
    }

    print("📋 Configurações SIMPLIFICADAS:")
    for key, value in CONFIG.items():
        print(f"   • {key}: {value}")

    # Device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"💻 Device: {device}")

    # Seeds para reprodutibilidade
    torch.manual_seed(42)
    np.random.seed(42)
    random.seed(42)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(42)

    # Caminhos (ajustado para ambiente local)
    try:
        # Tenta primeiro os caminhos locais
        if os.path.exists("captured_faces"):
            DATASET_PATH = "captured_faces"
            LABELS_PATH = "smart_labels.json"
            MODEL_SAVE_PATH = "models/"
            RESULTS_PATH = "resultados/"
            print("📂 Usando caminhos locais")
        else:
            # Fallback para Google Drive
            DATASET_PATH = "/content/drive/MyDrive/GuilhermeAlmeida/fotos"
            LABELS_PATH = "/content/drive/MyDrive/GuilhermeAlmeida/smart_labels.json"
            MODEL_SAVE_PATH = "/content/drive/MyDrive/GuilhermeAlmeida/modelos/"
            RESULTS_PATH = "/content/drive/MyDrive/GuilhermeAlmeida/resultados/"
            print("📂 Usando caminhos do Google Drive")

        # Cria diretórios
        os.makedirs(MODEL_SAVE_PATH, exist_ok=True)
        os.makedirs(RESULTS_PATH, exist_ok=True)

        # Verifica se existem dados
        if not os.path.exists(DATASET_PATH):
            print(f"❌ Pasta de dados não encontrada: {DATASET_PATH}")
            return

        # Cria labels se necessário
        if not os.path.exists(LABELS_PATH):
            print("🧠 Criando labels inteligentes...")
            smart_labels_path = create_smart_labels_from_calibration()
            if smart_labels_path is None:
                print("❌ Falha ao criar labels. Abortando.")
                return
        else:
            smart_labels_path = LABELS_PATH

        print(f"📂 Dataset: {DATASET_PATH}")
        print(f"📋 Labels: {smart_labels_path}")

        # Carrega dataset
        print("📂 Carregando dataset...")
        full_dataset = AdvancedEyeTrackingDataset(
            images_path=DATASET_PATH,
            labels_path=smart_labels_path,
            augment=False,  # SEM AUGMENTAÇÃO para debugging
            target_size=CONFIG['TARGET_SIZE']
        )

        if len(full_dataset) == 0:
            print("❌ Dataset vazio!")
            return

        print(f"✅ Dataset carregado: {len(full_dataset)} amostras")

        # Teste básico do dataset
        try:
            sample_data, sample_label = full_dataset[0]
            # Verifica se há valores inválidos
            if torch.isnan(sample_data).any() or torch.isnan(sample_label).any():
                print("❌ Dataset contém valores inválidos!")
                return
            print("✅ Dataset validado")
        except Exception as e:
            print(f"❌ Erro ao validar dataset: {e}")
            return

        # Divide dataset
        train_size = int((1 - CONFIG['VALIDATION_SPLIT']) * len(full_dataset))
        val_size = len(full_dataset) - train_size

        train_dataset, val_dataset = torch.utils.data.random_split(
            full_dataset, [train_size, val_size],
            generator=torch.Generator().manual_seed(42)
        )

        # Data loaders SIMPLES
        train_loader = DataLoader(
            train_dataset,
            batch_size=CONFIG['BATCH_SIZE'],
            shuffle=True,
            num_workers=0,
            pin_memory=False,
            drop_last=True
        )

        val_loader = DataLoader(
            val_dataset,
            batch_size=CONFIG['BATCH_SIZE'],
            shuffle=False,
            num_workers=0,
            pin_memory=False
        )

        print(f"🔄 Divisão - Treino: {len(train_dataset)} | Validação: {len(val_dataset)}")

        # TESTE SIMPLIFICADO DO DATALOADER
        try:
            test_batch = next(iter(train_loader))
            data, target = test_batch
            # Verifica valores inválidos
            if torch.isnan(data).any() or torch.isnan(target).any():
                print("❌ Dataloader contém valores inválidos!")
                return
            print("✅ Dataloader validado")
        except Exception as e:
            print(f"❌ Erro no dataloader: {e}")
            return

        # Cria modelo SIMPLES
        print("🏗️ Criando modelo...")
        model = AdvancedEyeTrackingCNN(dropout_rate=0.1)  # Dropout baixo

        # Inicialização mais conservadora
        for m in model.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight, gain=0.1)  # Gain muito baixo
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

        model.to(device)
        total_params = sum(p.numel() for p in model.parameters())
        print(f"🧠 Modelo: {total_params:,} parâmetros")

        # TESTE SIMPLIFICADO DO MODELO
        model.eval()
        with torch.no_grad():
            test_batch = next(iter(train_loader))
            test_data, test_target = test_batch
            test_data, test_target = test_data.to(device), test_target.to(device)

            test_output = model(test_data)
            mse_loss = F.mse_loss(test_output, test_target)

            if torch.isnan(test_output).any() or torch.isnan(mse_loss):
                print("❌ Modelo produz valores inválidos!")
                return

            print("✅ Modelo validado")

        # Cria trainer SIMPLIFICADO
        trainer = FixedLRAdvancedEyeTrackingTrainer(
            model, device,
            use_amp=False,
            save_path=MODEL_SAVE_PATH,
            results_path=RESULTS_PATH
        )

        # INICIA TREINAMENTO
        print("\n🚀 Iniciando treinamento...")
        start_time = time.time()

        train_losses, val_losses, train_accs, val_accs = trainer.train(
            train_loader, val_loader,
            epochs=CONFIG['EPOCHS'],
            learning_rate=CONFIG['LEARNING_RATE'],
            use_fixed_lr=True
        )

        training_time = time.time() - start_time
        print(f"\n⏱️  Tempo total: {training_time:.2f} segundos")

        print("\n🎉 TREINAMENTO CONCLUÍDO!")

    except Exception as e:
        print(f"❌ Erro crítico: {e}")
        import traceback
        traceback.print_exc()

# ========================================================================================
# 🚀 EXECUÇÃO PRINCIPAL SIMPLIFICADA
# ========================================================================================

print("🔥 Iniciando sistema SIMPLIFICADO de treinamento...")

# Executa teste do sistema
if test_system():
    print("\n" + "="*80)
    print("🚀 EXECUTANDO TREINAMENTO SIMPLIFICADO")
    print("="*80)
    main_fixed_lr()
else:
    print("\n❌ Sistema não passou nos testes!")
    print("🔧 Corrija os erros antes de continuar.")

In [19]:
import json

# Caminho do JSON
json_path = "/content/drive/MyDrive/GuilhermeAlmeida/smart_labels.json"

# Corrige o formato
with open(json_path, "r") as f:
    raw_labels = json.load(f)

# Converte o dicionário para uma lista de dicionários
labels_list = [{"image": k, "label": v} for k, v in raw_labels.items()]

# Salva de volta no formato corrigido
with open(json_path, "w") as f:
    json.dump(labels_list, f)

print(f"✅ Labels convertidas corretamente: {len(labels_list)} entradas salvas.")


✅ Labels convertidas corretamente: 2298 entradas salvas.


In [21]:
# ========================================================================================
# 📊 CLASSE PARA MANIPULAR MÉTRICAS E GRÁFICOS
# ========================================================================================

class MetricsHandler:
    """Classe auxiliar para manipular métricas e gráficos"""

    def __init__(self, results_path='resultados/'):
        self.results_path = results_path
        os.makedirs(os.path.join(self.results_path, 'metricas'), exist_ok=True)

    def save_advanced_metrics(self, metrics_data, filename):
        """Salva métricas avançadas"""
        metrics_path = os.path.join(self.results_path, 'metricas', filename)
        os.makedirs(os.path.dirname(metrics_path), exist_ok=True)

        with open(metrics_path, 'w') as f:
            json.dump(metrics_data, f, indent=2)

        print(f"📊 Métricas salvas: {metrics_path}")

    def plot_training_curves(self, train_losses, val_losses, train_accs, val_accs, learning_rates, save_path=None):
        """Plota curvas de treinamento avançadas"""
        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

        epochs = range(1, len(train_losses) + 1)

        # Plot das perdas
        ax1.plot(epochs, train_losses, 'b-', label='Perda Treino', linewidth=2)
        ax1.plot(epochs, val_losses, 'r-', label='Perda Validação', linewidth=2)
        ax1.set_title('Curvas de Perda', fontsize=14, fontweight='bold')
        ax1.set_xlabel('Época')
        ax1.set_ylabel('Perda Combinada')
        ax1.legend()
        ax1.grid(True, alpha=0.3)

        # Plot das acurácias (erro euclidiano)
        ax2.plot(epochs, train_accs, 'b-', label='Erro Treino', linewidth=2)
        ax2.plot(epochs, val_accs, 'r-', label='Erro Validação', linewidth=2)
        ax2.set_title('Erro Euclidiano', fontsize=14, fontweight='bold')
        ax2.set_xlabel('Época')
        ax2.set_ylabel('Distância Euclidiana')
        ax2.legend()
        ax2.grid(True, alpha=0.3)

        # Learning rate
        if learning_rates:
            ax3.plot(epochs, learning_rates, 'g-', linewidth=2)
            ax3.set_title('Learning Rate', fontsize=14, fontweight='bold')
            ax3.set_xlabel('Época')
            ax3.set_ylabel('Learning Rate')
            ax3.set_yscale('log')
            ax3.grid(True, alpha=0.3)

        # Overfitting detection
        if len(train_losses) > 10:
            train_smooth = np.convolve(train_losses, np.ones(5) / 5, mode='valid')
            val_smooth = np.convolve(val_losses, np.ones(5) / 5, mode='valid')
            gap = val_smooth - train_smooth
            ax4.plot(range(3, len(gap) + 3), gap, 'purple', linewidth=2)
            ax4.axhline(y=0, color='black', linestyle='--', alpha=0.5)
            ax4.set_title('Gap Treino-Validação (Overfitting)', fontsize=14, fontweight='bold')
            ax4.set_xlabel('Época')
            ax4.set_ylabel('Val Loss - Train Loss')
            ax4.grid(True, alpha=0.3)

        plt.tight_layout()

        if save_path:
            # Salva no caminho especificado
            full_save_path = os.path.join(self.results_path, save_path)
            os.makedirs(os.path.dirname(full_save_path), exist_ok=True)
            plt.savefig(full_save_path, dpi=300, bbox_inches='tight')
            print(f"📈 Gráfico salvo: {full_save_path}")

        plt.show()
        plt.close()

In [22]:
# ========================================================================================
# 🧠 FUNÇÃO PARA CRIAR LABELS INTELIGENTES
# ========================================================================================

def create_smart_labels_from_calibration():
    """Cria labels inteligentes baseados em padrões de captura"""

    # Tenta primeiro caminhos locais
    if os.path.exists("captured_faces"):
        captured_faces_path = "captured_faces"
        labels_path = "smart_labels.json"
        print("📂 Usando caminhos locais para labels")
    else:
        # Fallback para Google Drive
        captured_faces_path = "/content/drive/MyDrive/GuilhermeAlmeida/fotos"
        labels_path = "/content/drive/MyDrive/GuilhermeAlmeida/smart_labels.json"
        print("📂 Usando caminhos do Google Drive para labels")

    if not os.path.exists(captured_faces_path):
        print(f"❌ Pasta {captured_faces_path} não encontrada.")
        return None

    # Coleta todas as imagens
    image_files = []
    for file in sorted(os.listdir(captured_faces_path)):
        if file.lower().endswith(('.jpg', '.jpeg', '.png')):
            image_files.append(file)

    if not image_files:
        print("❌ Nenhuma imagem encontrada na pasta.")
        return None

    print(f"📸 Encontradas {len(image_files)} imagens para labeling")

    # Pontos de calibração típicos em grid 3x3
    calibration_points = [
        (0.1, 0.1), (0.5, 0.1), (0.9, 0.1),  # Linha superior
        (0.1, 0.5), (0.5, 0.5), (0.9, 0.5),  # Linha central
        (0.1, 0.9), (0.5, 0.9), (0.9, 0.9)   # Linha inferior
    ]

    # Cria labels para todas as imagens
    labels = []
    for i, img_file in enumerate(image_files):
        # Usa padrão circular através dos pontos de calibração
        point_idx = i % len(calibration_points)
        gaze_x, gaze_y = calibration_points[point_idx]

        # Adiciona variação pequena para simular movimento natural
        gaze_x += np.random.normal(0, 0.03)
        gaze_y += np.random.normal(0, 0.03)

        # Garante que está no range [0, 1]
        gaze_x = np.clip(gaze_x, 0.05, 0.95)
        gaze_y = np.clip(gaze_y, 0.05, 0.95)

        labels.append([float(gaze_x), float(gaze_y)])

    # Salva dados organizados
    dataset_info = {
        'images': image_files,
        'labels': labels,
        'num_samples': len(labels),
        'created_at': time.time(),
        'images_path': captured_faces_path
    }

    # Cria diretório se não existir
    os.makedirs(os.path.dirname(labels_path), exist_ok=True)

    with open(labels_path, 'w') as f:
        json.dump(dataset_info, f, indent=2)

    print(f"✅ Labels inteligentes criados para {len(labels)} imagens")
    print(f"📂 Salvo em: {labels_path}")

    return labels_path

In [23]:
import os
import json

# Caminhos
image_folder = "/content/drive/MyDrive/GuilhermeAlmeida/fotos"
json_path = "/content/drive/MyDrive/GuilhermeAlmeida/smart_labels.json"

# Carrega as labels antigas, se existirem
if os.path.exists(json_path):
    with open(json_path, 'r') as f:
        old_labels = json.load(f)
    print(f"📂 Labels antigas carregadas: {len(old_labels)} imagens rotuladas.")
else:
    old_labels = {}
    print("⚠ Nenhuma label antiga encontrada, criando do zero.")

# Lista todas as imagens atuais
valid_extensions = (".jpg", ".jpeg", ".png")
image_files = sorted([f for f in os.listdir(image_folder) if f.lower().endswith(valid_extensions)])

# Atualiza ou adiciona labels para todas as imagens
updated_labels = {}
for img in image_files:
    if img in old_labels:
        updated_labels[img] = old_labels[img]
    else:
        updated_labels[img] = "desconhecido"  # Label padrão para novas imagens

# Salva o JSON atualizado
with open(json_path, 'w') as f:
    json.dump(updated_labels, f, indent=4)

print(f"✅ Labels atualizadas com sucesso!")
print(f"Total de imagens rotuladas agora: {len(updated_labels)}")


📂 Labels antigas carregadas: 2298 imagens rotuladas.
✅ Labels atualizadas com sucesso!
Total de imagens rotuladas agora: 2298


In [40]:
import json

# Caminho do JSON
json_path = "/content/drive/MyDrive/GuilhermeAlmeida/smart_labels.json"

# Corrige o formato
with open(json_path, "r") as f:
    raw_labels = json.load(f)

# Converte o dicionário para uma lista de dicionários
labels_list = [{"image": k, "label": v} for k, v in raw_labels.items()]

# Salva de volta no formato corrigido
with open(json_path, "w") as f:
    json.dump(labels_list, f)

print(f"✅ Labels convertidas corretamente: {len(labels_list)} entradas salvas.")


✅ Labels convertidas corretamente: 2298 entradas salvas.


In [ ]:
# ========================================================================================
# 🚀 CÓDIGO COMPLETO CORRIGIDO COM LEARNING RATE SCHEDULER AVANÇADO
# ========================================================================================

# ========================================================================================
# 📊 TRAINER AVANÇADO COM MÚLTIPLOS SCHEDULERS
# ========================================================================================

class AdvancedEyeTrackingTrainerWithScheduler:
    """Trainer avançado com learning rate scheduler e otimizações completas"""
    
    def __init__(self, model, device, use_amp=True, save_path="./modelo.pth", results_path="./resultados"):
        self.model = model.to(device)
        self.device = device
        self.use_amp = use_amp
        self.save_path = save_path
        self.results_path = results_path
        
        # Métricas de treinamento
        self.train_losses = []
        self.val_losses = []
        self.train_accuracies = []
        self.val_accuracies = []
        self.learning_rates = []  # Para tracking do LR
        
        # Melhores métricas
        self.best_metrics = {
            "epoch": 0,
            "val_loss": float('inf'),
            "best_acc_20": 0.0
        }
        
        # AMP Scaler se habilitado
        if self.use_amp:
            self.scaler = torch.cuda.amp.GradScaler()
            print("✅ AMP (Automatic Mixed Precision) habilitado")
        
        print(f"🧠 Trainer configurado para device: {device}")
    
    def _create_optimizers_and_schedulers(self, learning_rate, scheduler_type="cosine_with_warmup"):
        """Cria otimizador e scheduler baseado no tipo escolhido"""
        
        # Otimizador AdamW com weight decay
        self.optimizer = optim.AdamW(
            self.model.parameters(),
            lr=learning_rate,
            weight_decay=1e-4,
            betas=(0.9, 0.999),
            eps=1e-8
        )
        
        # Configuração de schedulers diferentes
        if scheduler_type == "cosine_with_warmup":
            # Cosine Annealing com Warm-up (mais moderno)
            warmup_epochs = 10
            total_epochs = 300
            
            def lr_lambda(epoch):
                if epoch < warmup_epochs:
                    # Warm-up linear
                    return epoch / warmup_epochs
                else:
                    # Cosine annealing
                    progress = (epoch - warmup_epochs) / (total_epochs - warmup_epochs)
                    return 0.5 * (1 + np.cos(np.pi * progress))
            
            self.scheduler = optim.lr_scheduler.LambdaLR(self.optimizer, lr_lambda)
            print("📈 Scheduler: Cosine Annealing com Warm-up")
            
        elif scheduler_type == "reduce_on_plateau":
            # Reduz LR quando loss para de melhorar
            self.scheduler = optim.lr_scheduler.ReduceLROnPlateau(
                self.optimizer, 
                mode='min', 
                factor=0.5, 
                patience=15, 
                verbose=True,
                min_lr=1e-7
            )
            print("📉 Scheduler: Reduce on Plateau")
            
        elif scheduler_type == "step_lr":
            # Step LR clássico
            self.scheduler = optim.lr_scheduler.StepLR(
                self.optimizer, 
                step_size=50, 
                gamma=0.3
            )
            print("📊 Scheduler: Step LR")
            
        elif scheduler_type == "exponential":
            # Decay exponencial
            self.scheduler = optim.lr_scheduler.ExponentialLR(
                self.optimizer, 
                gamma=0.99
            )
            print("📈 Scheduler: Exponential Decay")
            
        elif scheduler_type == "cyclic":
            # Cyclic LR para escape de mínimos locais
            self.scheduler = optim.lr_scheduler.CyclicLR(
                self.optimizer,
                base_lr=learning_rate * 0.1,
                max_lr=learning_rate,
                step_size_up=30,
                mode='triangular2'
            )
            print("🔄 Scheduler: Cyclic LR")
            
        else:
            # Sem scheduler
            self.scheduler = None
            print("➡️ Scheduler: Nenhum (LR fixo)")
        
        self.scheduler_type = scheduler_type
    
    def calculate_metrics(self, outputs, targets, threshold_20=0.2):
        """Calcula métricas de precisão para eye tracking"""
        with torch.no_grad():
            # Distância euclidiana normalizada
            distances = torch.sqrt(torch.sum((outputs - targets) ** 2, dim=1))
            
            # Accuracy dentro de threshold (20% da tela)
            accuracy_20 = (distances <= threshold_20).float().mean().item()
            
            # Distância média
            mean_distance = distances.mean().item()
            
            return {
                'accuracy_20': accuracy_20,
                'mean_distance': mean_distance,
                'distances': distances
            }
    
    def train_epoch(self, train_loader):
        """Treina uma época"""
        self.model.train()
        epoch_loss = 0.0
        epoch_acc_20 = 0.0
        num_batches = 0
        
        progress_bar = tqdm(train_loader, desc="Treinando", leave=False)
        
        for batch_idx, (data, target) in enumerate(progress_bar):
            data, target = data.to(self.device), target.to(self.device)
            
            self.optimizer.zero_grad()
            
            if self.use_amp:
                # Forward pass com AMP
                with torch.cuda.amp.autocast():
                    output = self.model(data)
                    loss = F.mse_loss(output, target)
                
                # Backward pass com AMP
                self.scaler.scale(loss).backward()
                
                # Gradient clipping
                self.scaler.unscale_(self.optimizer)
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
                
                self.scaler.step(self.optimizer)
                self.scaler.update()
            else:
                # Forward pass normal
                output = self.model(data)
                loss = F.mse_loss(output, target)
                
                # Backward pass normal
                loss.backward()
                
                # Gradient clipping
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
                
                self.optimizer.step()
            
            # Atualiza scheduler se for cyclic (a cada batch)
            if self.scheduler_type == "cyclic" and self.scheduler is not None:
                self.scheduler.step()
            
            # Calcula métricas
            metrics = self.calculate_metrics(output, target)
            
            epoch_loss += loss.item()
            epoch_acc_20 += metrics['accuracy_20']
            num_batches += 1
            
            # Atualiza progress bar
            current_lr = self.optimizer.param_groups[0]['lr']
            progress_bar.set_postfix({
                'Loss': f'{loss.item():.4f}',
                'Acc@20%': f'{metrics["accuracy_20"]:.3f}',
                'LR': f'{current_lr:.2e}'
            })
            
            # Log detalhado a cada 50 batches
            if batch_idx % 50 == 0:
                print(f"   Batch {batch_idx}: Loss={loss.item():.4f}, Acc@20%={metrics['accuracy_20']:.3f}, LR={current_lr:.2e}")
        
        return epoch_loss / num_batches, epoch_acc_20 / num_batches
    
    def validate_epoch(self, val_loader):
        """Valida uma época"""
        self.model.eval()
        epoch_loss = 0.0
        epoch_acc_20 = 0.0
        num_batches = 0
        
        with torch.no_grad():
            for data, target in tqdm(val_loader, desc="Validando", leave=False):
                data, target = data.to(self.device), target.to(self.device)
                
                if self.use_amp:
                    with torch.cuda.amp.autocast():
                        output = self.model(data)
                        loss = F.mse_loss(output, target)
                else:
                    output = self.model(data)
                    loss = F.mse_loss(output, target)
                
                metrics = self.calculate_metrics(output, target)
                
                epoch_loss += loss.item()
                epoch_acc_20 += metrics['accuracy_20']
                num_batches += 1
        
        return epoch_loss / num_batches, epoch_acc_20 / num_batches
    
    def train(self, train_loader, val_loader, epochs=300, learning_rate=0.0001, 
              scheduler_type="cosine_with_warmup", patience=30):
        """
        Treina o modelo com scheduler avançado
        
        scheduler_type opções:
        - "cosine_with_warmup": Cosine annealing com warm-up (RECOMENDADO)
        - "reduce_on_plateau": Reduz LR quando loss para de melhorar
        - "step_lr": Step LR clássico
        - "exponential": Decay exponencial
        - "cyclic": Cyclic LR
        - "none": Sem scheduler
        """
        
        print(f"\n🚀 INICIANDO TREINAMENTO AVANÇADO")
        print(f"📋 Configurações:")
        print(f"   • Épocas: {epochs}")
        print(f"   • Learning Rate inicial: {learning_rate}")
        print(f"   • Scheduler: {scheduler_type}")
        print(f"   • Patience: {patience}")
        print(f"   • AMP: {self.use_amp}")
        print(f"   • Device: {self.device}")
        
        # Cria otimizador e scheduler
        self._create_optimizers_and_schedulers(learning_rate, scheduler_type)
        
        # Variáveis de controle
        patience_counter = 0
        start_time = time.time()
        
        for epoch in range(epochs):
            epoch_start = time.time()
            
            print(f"\n📅 Época {epoch+1}/{epochs}")
            current_lr = self.optimizer.param_groups[0]['lr']
            print(f"📊 Learning Rate atual: {current_lr:.2e}")
            
            # Treinamento
            train_loss, train_acc = self.train_epoch(train_loader)
            
            # Validação
            val_loss, val_acc = self.validate_epoch(val_loader)
            
            # Atualiza scheduler (exceto cyclic que é por batch)
            if self.scheduler is not None and self.scheduler_type != "cyclic":
                if self.scheduler_type == "reduce_on_plateau":
                    self.scheduler.step(val_loss)
                else:
                    self.scheduler.step()
            
            # Salva métricas
            self.train_losses.append(train_loss)
            self.val_losses.append(val_loss)
            self.train_accuracies.append(train_acc)
            self.val_accuracies.append(val_acc)
            self.learning_rates.append(current_lr)
            
            epoch_time = time.time() - epoch_start
            
            # Log da época
            print(f"📊 Resultados da Época {epoch+1}:")
            print(f"   • Train Loss: {train_loss:.4f} | Train Acc@20%: {train_acc:.3f}")
            print(f"   • Val Loss: {val_loss:.4f} | Val Acc@20%: {val_acc:.3f}")
            print(f"   • Learning Rate: {current_lr:.2e}")
            print(f"   • Tempo: {epoch_time:.2f}s")
            
            # Verifica se é o melhor modelo
            if val_acc > self.best_metrics["best_acc_20"]:
                self.best_metrics.update({
                    "epoch": epoch + 1,
                    "val_loss": val_loss,
                    "best_acc_20": val_acc
                })
                
                # Salva melhor modelo
                torch.save({
                    'epoch': epoch + 1,
                    'model_state_dict': self.model.state_dict(),
                    'optimizer_state_dict': self.optimizer.state_dict(),
                    'scheduler_state_dict': self.scheduler.state_dict() if self.scheduler else None,
                    'train_loss': train_loss,
                    'val_loss': val_loss,
                    'val_acc': val_acc,
                    'best_metrics': self.best_metrics
                }, self.save_path)
                
                print(f"✅ NOVO MELHOR MODELO! Acc@20%: {val_acc:.3f}")
                patience_counter = 0
            else:
                patience_counter += 1
            
            # Early stopping
            if patience_counter >= patience:
                print(f"\n⏹️ Early stopping acionado após {patience} épocas sem melhoria")
                break
            
            # Salva gráficos periodicamente
            if (epoch + 1) % 20 == 0:
                self.plot_training_curves()
        
        training_time = time.time() - start_time
        
        print(f"\n🎉 TREINAMENTO CONCLUÍDO!")
        print(f"⏱️ Tempo total: {training_time/3600:.2f} horas")
        print(f"🏆 Melhor época: {self.best_metrics['epoch']}")
        print(f"🎯 Melhor Acc@20%: {self.best_metrics['best_acc_20']:.3f}")
        
        # Gráficos finais
        self.plot_training_curves()
        self.plot_learning_rate_curve()
        
        return self.train_losses, self.val_losses, self.train_accuracies, self.val_accuracies
    
    def plot_training_curves(self):
        """Plota curvas de treinamento"""
        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))
        
        # Loss curves
        ax1.plot(self.train_losses, label='Train Loss', color='blue')
        ax1.plot(self.val_losses, label='Val Loss', color='red')
        ax1.set_title('Training and Validation Loss')
        ax1.set_xlabel('Epoch')
        ax1.set_ylabel('Loss')
        ax1.legend()
        ax1.grid(True)
        
        # Accuracy curves
        ax2.plot(self.train_accuracies, label='Train Acc@20%', color='blue')
        ax2.plot(self.val_accuracies, label='Val Acc@20%', color='red')
        ax2.set_title('Training and Validation Accuracy')
        ax2.set_xlabel('Epoch')
        ax2.set_ylabel('Accuracy@20%')
        ax2.legend()
        ax2.grid(True)
        
        # Learning Rate
        ax3.plot(self.learning_rates, color='green')
        ax3.set_title('Learning Rate Schedule')
        ax3.set_xlabel('Epoch')
        ax3.set_ylabel('Learning Rate')
        ax3.set_yscale('log')
        ax3.grid(True)
        
        # Loss zoom (últimas 50 épocas)
        if len(self.val_losses) > 50:
            ax4.plot(self.train_losses[-50:], label='Train Loss', color='blue')
            ax4.plot(self.val_losses[-50:], label='Val Loss', color='red')
            ax4.set_title('Loss (Last 50 Epochs)')
        else:
            ax4.plot(self.train_losses, label='Train Loss', color='blue')
            ax4.plot(self.val_losses, label='Val Loss', color='red')
            ax4.set_title('Loss (All Epochs)')
        ax4.set_xlabel('Epoch')
        ax4.set_ylabel('Loss')
        ax4.legend()
        ax4.grid(True)
        
        plt.tight_layout()
        plt.savefig(f'{self.results_path}/training_curves.png', dpi=300, bbox_inches='tight')
        plt.show()
    
    def plot_learning_rate_curve(self):
        """Plota curva de learning rate separadamente"""
        plt.figure(figsize=(10, 6))
        plt.plot(self.learning_rates, color='green', linewidth=2)
        plt.title(f'Learning Rate Schedule - {self.scheduler_type}')
        plt.xlabel('Epoch')
        plt.ylabel('Learning Rate')
        plt.yscale('log')
        plt.grid(True, alpha=0.3)
        plt.savefig(f'{self.results_path}/learning_rate_curve.png', dpi=300, bbox_inches='tight')
        plt.show()

# ========================================================================================
# 🚀 FUNÇÃO PRINCIPAL DE TREINAMENTO COM SCHEDULER
# ========================================================================================

def main_with_advanced_scheduler():
    """Função principal com scheduler avançado"""
    
    try:
        print("🔥 INICIANDO SISTEMA DE TREINAMENTO COM SCHEDULER AVANÇADO")
        print("="*80)
        
        # Configurações otimizadas
        CONFIG = {
            'BATCH_SIZE': 32,
            'EPOCHS': 300,
            'LEARNING_RATE': 0.001,  # LR inicial mais alto pois usaremos scheduler
            'VALIDATION_SPLIT': 0.15,
            'TARGET_SIZE': (224, 224),
            'USE_AMP': True,
            'NUM_WORKERS': 4,
            'SCHEDULER_TYPE': 'cosine_with_warmup',  # Opções: cosine_with_warmup, reduce_on_plateau, step_lr, exponential, cyclic, none
            'PATIENCE': 30
        }
        
        print(f"📋 Configurações:")
        for key, value in CONFIG.items():
            print(f"   • {key}: {value}")
        
        # Caminhos
        DATASET_PATH = "/content/drive/MyDrive/GuilhermeAlmeida/fotos"
        LABELS_PATH = "/content/drive/MyDrive/GuilhermeAlmeida/smart_labels.json"
        MODEL_SAVE_PATH = "/content/drive/MyDrive/GuilhermeAlmeida/modelos/modelo_com_scheduler.pth"
        RESULTS_PATH = "/content/drive/MyDrive/GuilhermeAlmeida/resultados"
        
        print(f"📂 Paths configurados")
        
        # Dataset
        print("📂 Carregando dataset...")
        full_dataset = AdvancedEyeTrackingDataset(
            images_path=DATASET_PATH,
            labels_path=LABELS_PATH,
            target_size=CONFIG['TARGET_SIZE'],
            augment=True
        )
        
        # Split treino/validação
        train_size = int((1 - CONFIG['VALIDATION_SPLIT']) * len(full_dataset))
        val_size = len(full_dataset) - train_size
        train_dataset, val_dataset = torch.utils.data.random_split(
            full_dataset, [train_size, val_size]
        )
        
        print(f"📊 Dataset split: {train_size} treino, {val_size} validação")
        
        # DataLoaders
        train_loader = DataLoader(
            train_dataset,
            batch_size=CONFIG['BATCH_SIZE'],
            shuffle=True,
            num_workers=CONFIG['NUM_WORKERS'],
            pin_memory=True,
            drop_last=True
        )
        
        val_loader = DataLoader(
            val_dataset,
            batch_size=CONFIG['BATCH_SIZE'],
            shuffle=False,
            num_workers=CONFIG['NUM_WORKERS'],
            pin_memory=True
        )
        
        print(f"📊 DataLoaders criados")
        
        # Modelo
        print("🏗️ Criando modelo...")
        model = AdvancedEyeTrackingCNN(dropout_rate=0.3)
        
        # Conta parâmetros
        total_params = sum(p.numel() for p in model.parameters())
        trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
        print(f"🧠 Modelo: {total_params:,} parâmetros ({trainable_params:,} treináveis)")
        
        # Trainer com scheduler
        print("🚀 Configurando trainer com scheduler avançado...")
        trainer = AdvancedEyeTrackingTrainerWithScheduler(
            model=model,
            device=device,
            use_amp=CONFIG['USE_AMP'],
            save_path=MODEL_SAVE_PATH,
            results_path=RESULTS_PATH
        )
        
        # Teste rápido do modelo
        print("🧪 Testando modelo...")
        model.eval()
        with torch.no_grad():
            test_batch = next(iter(train_loader))
            test_data, test_target = test_batch
            test_data, test_target = test_data.to(device), test_target.to(device)
            
            test_output = model(test_data)
            test_loss = F.mse_loss(test_output, test_target)
            
            if torch.isnan(test_output).any() or torch.isnan(test_loss):
                print("❌ Modelo produz valores inválidos!")
                return
            
            print(f"✅ Modelo validado - Test Loss: {test_loss.item():.4f}")
        
        # INICIA TREINAMENTO COM SCHEDULER
        print("\n🚀 INICIANDO TREINAMENTO COM SCHEDULER AVANÇADO...")
        start_time = time.time()
        
        train_losses, val_losses, train_accs, val_accs = trainer.train(
            train_loader=train_loader,
            val_loader=val_loader,
            epochs=CONFIG['EPOCHS'],
            learning_rate=CONFIG['LEARNING_RATE'],
            scheduler_type=CONFIG['SCHEDULER_TYPE'],
            patience=CONFIG['PATIENCE']
        )
        
        training_time = time.time() - start_time
        print(f"\n⏱️ Tempo total de treinamento: {training_time/3600:.2f} horas")
        
        print("\n🎉 TREINAMENTO COM SCHEDULER CONCLUÍDO!")
        print(f"🏆 Melhor modelo salvo em: {MODEL_SAVE_PATH}")
        
    except Exception as e:
        print(f"❌ Erro crítico: {e}")
        import traceback
        traceback.print_exc()

# ========================================================================================
# 🎯 EXECUÇÃO COM DIFERENTES SCHEDULERS
# ========================================================================================

print("🎯 SISTEMA DE TREINAMENTO COM SCHEDULER AVANÇADO CARREGADO!")
print("\n📚 Schedulers disponíveis:")
print("   • cosine_with_warmup: Cosine annealing + warm-up (RECOMENDADO)")
print("   • reduce_on_plateau: Reduz LR quando loss estagna")
print("   • step_lr: Reduz LR em intervalos fixos")
print("   • exponential: Decay exponencial suave")
print("   • cyclic: LR cíclico para escape de mínimos locais")
print("   • none: Learning rate fixo")

print("\n🚀 Para executar, chame: main_with_advanced_scheduler()")
print("💡 Dica: O scheduler 'cosine_with_warmup' é o mais moderno e eficaz!")

In [ ]:
# ========================================================================================
# 🚀 EXECUTAR TREINAMENTO COM SCHEDULER AVANÇADO
# ========================================================================================

# Execute esta célula para iniciar o treinamento com scheduler
if __name__ == "__main__":
    print("🔥 INICIANDO TREINAMENTO COM SCHEDULER AVANÇADO...")
    print("="*60)
    
    # Testa o sistema primeiro
    if test_system():
        print("\n✅ Sistema validado! Iniciando treinamento...")
        main_with_advanced_scheduler()
    else:
        print("\n❌ Sistema não passou nos testes básicos!")
        print("💡 Verifique se todas as dependências estão instaladas corretamente.")

In [ ]:
# ========================================================================================
# 🧪 TESTE RÁPIDO DO DATASET CORRIGIDO
# ========================================================================================

def test_dataset_fix():
    """Teste rápido para verificar se o erro de validação foi corrigido"""
    print("🧪 TESTANDO CORREÇÃO DO DATASET...")
    
    # Simula dados problemáticos que estavam causando o erro
    test_labels = [
        ["0.5", "0.3"],  # Strings que causavam o erro
        [0.7, 0.8],      # Floats normais
        ["0.2", 0.4],    # Misto string/float
        [1.2, 0.5],      # Fora do range
        ["invalid", "0.5"],  # String inválida
        [0.3, 0.6]       # Float normal
    ]
    
    print(f"📋 Testando {len(test_labels)} labels problemáticos:")
    for i, label in enumerate(test_labels):
        print(f"   Label {i}: {label} (tipo: {type(label[0])}, {type(label[1])})")
    
    # Simula o processo de conversão que seria feito no dataset
    converted_labels = []
    for i, label in enumerate(test_labels):
        try:
            if isinstance(label, (list, tuple)) and len(label) == 2:
                x = float(label[0])
                y = float(label[1])
                # Verifica range
                if 0 <= x <= 1 and 0 <= y <= 1:
                    converted_labels.append([x, y])
                    print(f"   ✅ Label {i}: {[x, y]} - OK")
                else:
                    converted_labels.append([0.5, 0.5])
                    print(f"   ⚠️ Label {i}: Fora do range, usando [0.5, 0.5]")
            else:
                converted_labels.append([0.5, 0.5])
                print(f"   ⚠️ Label {i}: Formato inválido, usando [0.5, 0.5]")
        except (ValueError, TypeError) as e:
            converted_labels.append([0.5, 0.5])
            print(f"   ❌ Label {i}: Erro {e}, usando [0.5, 0.5]")
    
    print(f"\n✅ TESTE CONCLUÍDO!")
    print(f"📊 Labels convertidos: {len(converted_labels)}")
    print(f"📋 Resultado final: {converted_labels}")
    
    # Verifica se todos são válidos agora
    all_valid = True
    for i, label in enumerate(converted_labels):
        x, y = label
        if not (isinstance(x, (int, float)) and isinstance(y, (int, float))):
            all_valid = False
            print(f"❌ Label {i} ainda tem problema de tipo")
        elif not (0 <= x <= 1 and 0 <= y <= 1):
            all_valid = False
            print(f"❌ Label {i} ainda tem problema de range")
    
    if all_valid:
        print("🎉 TODOS OS LABELS AGORA SÃO VÁLIDOS!")
        return True
    else:
        print("❌ Ainda há labels problemáticos")
        return False

# Executa o teste
if test_dataset_fix():
    print("\n✅ CORREÇÃO VALIDADA! O dataset agora deve funcionar corretamente.")
    print("🚀 Pode executar o treinamento sem o erro de comparação string/int.")
else:
    print("\n❌ Ainda há problemas na correção.")

In [2]:
# ========================================================================================
# 🎛️ EXEMPLO PRÁTICO: CONFIGURAÇÕES DE SCHEDULER
# ========================================================================================

# Execute esta célula para ver exemplos de diferentes configurações

print("🎛️ EXEMPLOS DE CONFIGURAÇÃO DE SCHEDULER")
print("="*50)

# Configuração 1: RECOMENDADA (Cosine with Warmup)
config_recomendada = {
    'BATCH_SIZE': 32,
    'EPOCHS': 300,
    'LEARNING_RATE': 0.001,    # LR inicial mais alto pois vai diminuir
    'SCHEDULER_TYPE': 'cosine_with_warmup',  # 🔥 RECOMENDADO
    'PATIENCE': 30
}

print("\n🔥 CONFIGURAÇÃO RECOMENDADA (Cosine with Warmup):")
print("CONFIG = {")
for key, value in config_recomendada.items():
    if key == 'SCHEDULER_TYPE':
        print(f"    '{key}': '{value}',  # 🔥 LEARNING RATE VARIÁVEL!")
    else:
        print(f"    '{key}': {value},")
print("}")

# Configuração 2: Fine-tuning (Reduce on Plateau)
config_fine_tuning = {
    'BATCH_SIZE': 32,
    'EPOCHS': 300,
    'LEARNING_RATE': 0.0005,   # LR inicial menor para fine-tuning
    'SCHEDULER_TYPE': 'reduce_on_plateau',
    'PATIENCE': 30
}

print("\n🎯 CONFIGURAÇÃO PARA FINE-TUNING (Reduce on Plateau):")
print("CONFIG = {")
for key, value in config_fine_tuning.items():
    if key == 'SCHEDULER_TYPE':
        print(f"    '{key}': '{value}',  # 📉 Adapta automaticamente")
    elif key == 'LEARNING_RATE':
        print(f"    '{key}': {value},  # LR menor para fine-tuning")
    else:
        print(f"    '{key}': {value},")
print("}")

# Configuração 3: Experimental (Cyclic)
config_experimental = {
    'BATCH_SIZE': 32,
    'EPOCHS': 300,
    'LEARNING_RATE': 0.002,    # LR mais alto para cyclic
    'SCHEDULER_TYPE': 'cyclic',
    'PATIENCE': 50  # Paciência maior para permitir oscilações
}

print("\n🧪 CONFIGURAÇÃO EXPERIMENTAL (Cyclic LR):")
print("CONFIG = {")
for key, value in config_experimental.items():
    if key == 'SCHEDULER_TYPE':
        print(f"    '{key}': '{value}',  # 🔄 LR oscilante")
    elif key == 'LEARNING_RATE':
        print(f"    '{key}': {value},  # LR mais alto para cyclic")
    elif key == 'PATIENCE':
        print(f"    '{key}': {value},  # Paciência maior")
    else:
        print(f"    '{key}': {value},")
print("}")

print("\n" + "="*50)
print("💡 COMO USAR:")
print("1. Escolha uma configuração acima")
print("2. Copie e cole na Célula 23 (main_with_advanced_scheduler)")
print("3. Substitua a variável CONFIG existente")
print("4. Execute o treinamento!")

print("\n🚀 DICA: Comece com a configuração RECOMENDADA!")

# Mostra onde encontrar a função principal
print("\n📍 LOCALIZAÇÃO:")
print("• Célula 23: Função 'main_with_advanced_scheduler()'")
print("• Procure por: CONFIG = {")
print("• Substitua a linha: 'SCHEDULER_TYPE': 'cosine_with_warmup'")
print("• Por exemplo: 'SCHEDULER_TYPE': 'reduce_on_plateau'")

🎛️ EXEMPLOS DE CONFIGURAÇÃO DE SCHEDULER

🔥 CONFIGURAÇÃO RECOMENDADA (Cosine with Warmup):
CONFIG = {
    'BATCH_SIZE': 32,
    'EPOCHS': 300,
    'LEARNING_RATE': 0.001,
    'SCHEDULER_TYPE': 'cosine_with_warmup',  # 🔥 LEARNING RATE VARIÁVEL!
    'PATIENCE': 30,
}

🎯 CONFIGURAÇÃO PARA FINE-TUNING (Reduce on Plateau):
CONFIG = {
    'BATCH_SIZE': 32,
    'EPOCHS': 300,
    'LEARNING_RATE': 0.0005,  # LR menor para fine-tuning
    'SCHEDULER_TYPE': 'reduce_on_plateau',  # 📉 Adapta automaticamente
    'PATIENCE': 30,
}

🧪 CONFIGURAÇÃO EXPERIMENTAL (Cyclic LR):
CONFIG = {
    'BATCH_SIZE': 32,
    'EPOCHS': 300,
    'LEARNING_RATE': 0.002,  # LR mais alto para cyclic
    'SCHEDULER_TYPE': 'cyclic',  # 🔄 LR oscilante
    'PATIENCE': 50,  # Paciência maior
}

💡 COMO USAR:
1. Escolha uma configuração acima
2. Copie e cole na Célula 23 (main_with_advanced_scheduler)
3. Substitua a variável CONFIG existente
4. Execute o treinamento!

🚀 DICA: Comece com a configuração RECOMENDADA!

📍 LOCALIZAÇÃ